In [ ]:
import gdown
gdown.download('http://bit.ly/3RhoNho','ns_202104_csv',quiet=False)
import pandas as pd
ns_df = pd.read_csv('ns_202104_csv',low_memory=False)
print(ns_df.head())
# 열 삭제하기
# unnamed:13은 필요없으니 삭제해보기
ns_book = ns_df.loc[:,'번호':'등록일자'] # 행은 모두, 열은 번호~등록일자까지 선택
print(ns_book.head())
print()

# 불리언 배열 이용
print(ns_df.columns) # 열이름 저장
print(ns_df.columns[0])
print()

selected_columns = ns_df.columns != 'Unnamed: 13' # 불리언 배열(넘파이 배열)로 저장하고
ns_book = ns_df.loc[:,selected_columns] # 해당 불리언 배열을 loc안에 넣는다.
print(ns_book.head()) # unnamed 없음
print()

# drop(삭제하려는 열 이름, axis(0 : 행, 1 : 열))
ns_book = ns_df.drop('Unnamed: 13', axis=1)
print(ns_book.head())
print(ns_df.head())
print()

# 여러개 삭제할 경우 []리스트 형식을 이용한다.
ns_book = ns_df.drop(['Unnamed: 13','부가기호'], axis=1)
print(ns_book.head())
print()

# inplace = True로 다른 변수에 저장할 필요 없이 바로 바꿀 수 있다.
ns_book.drop('주제분류번호',axis=1,inplace=True)
print(ns_book.head())
print()

# dropna(axis=1(열 기준)) : NaN이 하나 이상 포함된 행이나 열 모두삭제
ns_book = ns_df.dropna(axis=1)
print(ns_book.head())
print()

# 모든 값이 NaN인 열 삭제 : how='all'
ns_book = ns_df.dropna(axis=1,how='all')
print(ns_book.head())

# 행 삭제하기
ns_book2 = ns_book.drop([0,1]) # 인덱스 0~1까지의 행 삭제
print(ns_book2.head())
ns_book2 = ns_book[:2] # 위와 같음

# 불리언 배열 이용
selected_rows = ns_df['출판사'] == '한빛미디어'
ns_book2 = ns_book[selected_rows]
print(ns_book2.head())
# loc[]에도 불리언 배열 사용 가능
# ns_book2 = ns_book.loc[selected_rows]는 위와 같다.

ns_book2 = ns_book[ns_book['대출건수']>1000] # ns_book의 대출건수가 1000보다 큰 불리언 배열을 넣고 돌림
print(ns_book2.head())

# 중복된 행 찾기 sum() : true = 1,false = 0으로 처리함,duplicated() 이용
sum(ns_book.duplicated()) # 0(False)이면 중복없음
sum(ns_book.duplicated(subset=['도서명','저자','ISBN'])) # 도서명, 저자, isbn 기준으로 찾으면 중복이 20000개가 넘음

#keep = False는 중복된 모든 행을 True로 표시한 불리언 배열 반환
dup_rows = ns_book.duplicated(subset=['도서명','저자','ISBN'],keep=False)
ns_book3 = ns_book[dup_rows]
print(ns_book3.head())

# 그룹별로 모으기
count_df = ns_book[['도서명','저자','ISBN','권','대출건수']] # 그룹으로 묶을 기준 열
group_df = count_df.groupby(by=['도서명', '저자','ISBN','권'],dropna=False) # 그룹으로 묶음 dropna=false NaN이 있는 값 미삭제(기본적으로 삭제한다.)
loan_count = group_df.sum()

# 위의 두 줄을 한번에 쓰면 loan_count = count_df.groupby(by=['도서명', '저자','ISBN','권'],dropna=False).sum()
print(loan_count.head())

# 원본 데이터 업데이트하기
dup_rows = ns_book.duplicated(subset=['도서명','저자','ISBN'],keep=False) # 중복된 행 = True인 배열
unique_rows = ~dup_rows # 중복된 행이 False
ns_book3 = ns_book[unique_rows].copy() # copy()를 안 쓰면 ns_book3와 ns_book[unique_rows]가 같은 곳을 가리켜서 값을 변경하면 안된다.
sum(ns_book3.duplicated(subset=['도서명','저자','ISBN','권'])) # 중복된 행이 없으니 0 출력

# 원본 데이터프레임 인덱스 설정하기
ns_book3.set_index(['도서명','저자','ISBN','권'],inplace=True) # loan_count와 같은 열이 되었다.
ns_book3.head()

# 업데이트(update()) ns_book3 대출건수 열을 loan_count 대출건수 열로 업데이트
ns_book3.update(loan_count) # loan_count와 같은 열 순서로 되어있으니 4개의 열에 해당하는 값들을 loan_count로 업데이트 한다.
ns_book3.head()

ns_book4 = ns_book3.reset_index() # 인덱스 열이 생긴걸 해제한다.
ns_book4.head()

print(sum(ns_book['대출건수']>100)) # 2311
print(sum(ns_book4['대출건수']>100)) # 2550 (중복된 도서의 대출건수를 합쳤기 때문이다.)

ns_book4 = ns_book4[ns_book.columns] # 열들을 update하고 set_index하는 과정에서 열 순서가 달라졌으므로 초기 열과 맞추기 위해 ns_book.columns를 이용한다.
ns_book4.head()
ns_book4.to_csv('ns_book.csv',index=False)

#일괄 처리용 함수
def data_cleaning(filename):
    """
    남산 도서관 장서 CSV 데이터 전처리 함수

    :param filename: CSV 파일이름
    """
    # 파일을 데이터프레임으로 읽습니다.
    ns_df = pd.read_csv(filename, low_memory=False)
    # NaN인 열을 삭제합니다.
    ns_book = ns_df.dropna(axis=1, how='all')

    # 대출건수를 합치기 위해 필요한 행만 추출하여 count_df 데이터프레임을 만듭니다.
    count_df = ns_book[['도서명','저자','ISBN','권','대출건수']]
    # 도서명, 저자, ISBN, 권을 기준으로 대출건수를 groupby합니다.
    loan_count = count_df.groupby(by=['도서명','저자','ISBN','권'], dropna=False).sum()
    # 원본 데이터프레임에서 중복된 행을 제외하고 고유한 행만 추출하여 복사합니다.
    dup_rows = ns_book.duplicated(subset=['도서명','저자','ISBN','권']) # 중복된것 = True
    unique_rows = ~dup_rows # 중복된것 = False
    ns_book3 = ns_book[unique_rows].copy()
    # 도서명, 저자, ISBN, 권을 인덱스로 설정합니다.
    ns_book3.set_index(['도서명','저자','ISBN','권'], inplace=True)
    # load_count에 저장된 누적 대출건수를 업데이트합니다.
    ns_book3.update(loan_count)

    # 인덱스를 재설정합니다.
    ns_book4 = ns_book3.reset_index()
    # 원본 데이터프레임의 열 순서로 변경합니다.
    ns_book4 = ns_book4[ns_book.columns]

    return ns_book4
new_ns_book4 = data_cleaning('ns_202104.csv')
ns_book4.equals(new_ns_book4) # True

Downloading...
From: http://bit.ly/3RhoNho
To: /content/ns_202104_csv
100%|██████████| 57.6M/57.6M [00:00<00:00, 80.0MB/s]


   번호                    도서명                저자    출판사  발행년도           ISBN  \
0   1                인공지능과 흙            김동훈 지음    민음사  2021  9788937444319   
1   2           가짜 행복 권하는 사회            김태형 지음   갈매나무  2021  9791190123969   
2   3  나도 한 문장 잘 쓰면 바랄 게 없겠네            김선영 지음   블랙피쉬  2021  9788968332982   
3   4                예루살렘 해변  이도 게펜 지음, 임재희 옮김  문학세계사  2021  9788970759906   
4   5  김성곤의 중국한시기행 : 장강·황하 편            김성곤 지음    김영사  2021  9788934990833   

  세트 ISBN 부가기호    권 주제분류번호  도서권수  대출건수        등록일자  Unnamed: 13  
0     NaN  NaN  NaN    NaN     1     0  2021-03-19          NaN  
1     NaN  NaN  NaN    NaN     1     0  2021-03-19          NaN  
2     NaN  NaN  NaN    NaN     1     0  2021-03-19          NaN  
3     NaN  NaN  NaN    NaN     1     0  2021-03-19          NaN  
4     NaN  NaN  NaN    NaN     1     0  2021-03-19          NaN  
   번호                    도서명                저자    출판사  발행년도           ISBN  \
0   1                인공지능과 흙            김동훈 지음    민음사  20

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈 지음,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형 지음,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,9788934990833,NaN,5,김영사,2021,NaN,NaN,NaN,1,0,2021-03-19
